# Reports and Interpretation Tutorial

In this tutorial, you'll learn how to:

1. Generate and save HTML reports
2. Interpret report visualizations
3. Extract data from reports
4. Share reports with stakeholders
5. Use reports for decision-making

**Time**: ~15 minutes  
**Prerequisites**: Complete Tutorials 01 and 02

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import json

from justiceai import audit, FairnessEvaluator

np.random.seed(42)
print("✓ Setup complete!")

## Part 1: Creating a Comprehensive Report

Let's create a realistic scenario and generate a full report.

In [ ]:
# Create realistic loan approval dataset
X, y = make_classification(
    n_samples=1500,
    n_features=12,
    n_informative=10,
    n_classes=2,
    weights=[0.65, 0.35],
    random_state=42
)

# Create DataFrame with meaningful feature names
feature_names = [
    'income', 'credit_score', 'employment_years', 'debt_ratio',
    'savings', 'property_value', 'loan_amount', 'age',
    'education_level', 'marital_status', 'dependents', 'job_stability'
]
df = pd.DataFrame(X, columns=feature_names)
df['approved'] = y

# Add sensitive attribute with bias
gender = np.random.choice(['Male', 'Female'], size=1500, p=[0.53, 0.47])
# Introduce historical bias
male_mask = gender == 'Male'
bias_indices = male_mask & (np.random.random(1500) < 0.15)
df.loc[bias_indices, 'approved'] = 1

df['gender'] = gender

print(f"Dataset created: {df.shape}")
print(f"\nApproval rate by gender:")
print(df.groupby('gender')['approved'].mean())

In [ ]:
# Train model
X = df[feature_names]
y = df['approved']
gender = df['gender']

X_train, X_test, y_train, y_test, gender_train, gender_test = train_test_split(
    X, y, gender, test_size=0.25, random_state=42, stratify=y
)

model = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42)
model.fit(X_train, y_train)

print("✓ Model trained!")

## Part 2: Generating the Report

Create a complete fairness report with all metrics and visualizations.

In [ ]:
# Generate comprehensive report
report = audit(
    model=model,
    X=X_test,
    y_true=y_test,
    sensitive_attrs=gender_test,
    fairness_threshold=0.05,
    output_path='loan_fairness_report.html'
)

print("✓ Report generated: loan_fairness_report.html")
print("\nOpen this file in your browser to see:")
print("  - Overall fairness score")
print("  - Metric breakdowns by group")
print("  - Interactive visualizations")
print("  - Actionable recommendations")

## Part 3: Interpreting Report Sections

### 3.1 Overall Fairness Score

The overall score (0-100) summarizes model fairness across all metrics.

In [ ]:
score = report.get_overall_score()

print(f"Overall Fairness Score: {score:.1f}/100\n")

# Interpretation guide
if score >= 90:
    print("🟢 EXCELLENT: Model exhibits very good fairness")
    print("   Action: Safe for production deployment")
elif score >= 70:
    print("🟡 GOOD: Model has minor fairness issues")
    print("   Action: Review violations, consider improvements")
elif score >= 50:
    print("🟠 MODERATE: Model has significant fairness concerns")
    print("   Action: Address violations before deployment")
else:
    print("🔴 POOR: Model has severe fairness problems")
    print("   Action: Major revisions required")

### 3.2 Fairness Violations

Identify specific metrics that failed the fairness threshold.

In [ ]:
# Get list of violations
issues = report.get_issues()

print(f"Found {len(issues)} fairness violation(s):\n")

if len(issues) > 0:
    for i, issue in enumerate(issues, 1):
        print(f"Issue {i}: {issue['metric']}")
        print(f"  Message: {issue['message']}")
        print(f"  Severity: {issue['severity']}")
        print(f"  Impact: {issue.get('impact', 'N/A')}")
        print()
else:
    print("✅ No violations! Model meets fairness criteria.")

### 3.3 Summary Statistics

Get detailed metrics and group-level statistics.

In [ ]:
summary = report.get_summary()

print("Summary Statistics:\n")
print(f"Overall Score: {summary['overall_score']:.1f}/100")
print(f"Passes Fairness: {summary['passes_fairness']}")
print(f"Number of Violations: {summary['n_violations']}")
print(f"\nKey Fairness Metrics:")
print(f"  Statistical Parity Diff: {summary['statistical_parity_diff']:.4f}")
print(f"  Disparate Impact Ratio: {summary['disparate_impact_ratio']:.4f}")

# Interpretation
print("\nInterpretation:")
if abs(summary['statistical_parity_diff']) < 0.05:
    print("  ✓ Statistical parity is satisfied")
else:
    diff = summary['statistical_parity_diff']
    advantaged = 'Males' if diff > 0 else 'Females'
    print(f"  ✗ {advantaged} receive more favorable outcomes")

if summary['disparate_impact_ratio'] >= 0.80:
    print("  ✓ Passes 80% rule (legal standard)")
else:
    print(f"  ✗ Fails 80% rule ({summary['disparate_impact_ratio']:.2f} < 0.80)")

## Part 4: Extracting Data for Further Analysis

Export report data in various formats for downstream use.

In [ ]:
# Export summary as JSON
summary_json = json.dumps(summary, indent=2, default=str)

# Save to file
with open('fairness_summary.json', 'w') as f:
    f.write(summary_json)

print("✓ Summary exported to fairness_summary.json")
print("\nSample JSON output:")
print(json.dumps(summary, indent=2, default=str)[:500] + "...")

In [ ]:
# Create a fairness metrics DataFrame
metrics_df = pd.DataFrame([
    {
        'Metric': 'Overall Score',
        'Value': f"{summary['overall_score']:.1f}",
        'Status': '✅' if summary['passes_fairness'] else '⚠️'
    },
    {
        'Metric': 'Statistical Parity Diff',
        'Value': f"{summary['statistical_parity_diff']:.4f}",
        'Status': '✅' if abs(summary['statistical_parity_diff']) < 0.05 else '⚠️'
    },
    {
        'Metric': 'Disparate Impact',
        'Value': f"{summary['disparate_impact_ratio']:.4f}",
        'Status': '✅' if summary['disparate_impact_ratio'] >= 0.80 else '⚠️'
    }
])

# Save as CSV
metrics_df.to_csv('fairness_metrics.csv', index=False)

print("✓ Metrics exported to fairness_metrics.csv")
print("\nMetrics table:")
print(metrics_df.to_string(index=False))

## Part 5: Using Reports for Decision Making

Practical workflows for different stakeholders.

### 5.1 For Data Scientists: Model Iteration

In [ ]:
# Compare multiple models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42)
}

evaluator = FairnessEvaluator(fairness_threshold=0.05)
results = []

print("Comparing models:\n")

for name, model in models.items():
    model.fit(X_train, y_train)
    
    # Quick fairness check
    metrics = evaluator.quick_check(model, X_test, y_test, gender_test)
    
    # Accuracy
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(y_test, model.predict(X_test))
    
    results.append({
        'Model': name,
        'Accuracy': f"{acc:.3f}",
        'Fairness Score': f"{metrics['overall_score']:.1f}",
        'Passes': '✅' if metrics['passes_fairness'] else '❌'
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))
print("\n💡 Choose the model with best balance of accuracy and fairness!")

### 5.2 For ML Engineers: Production Monitoring

In [ ]:
# Simulating production monitoring
def monitor_model_fairness(model, data_batch, threshold=70):
    """Monitor model fairness on new data batches."""
    X, y, gender = data_batch
    
    evaluator = FairnessEvaluator()
    metrics = evaluator.quick_check(model, X, y, gender)
    
    score = metrics['overall_score']
    
    if score < threshold:
        return {
            'status': 'ALERT',
            'score': score,
            'message': f'Fairness score dropped to {score:.1f}',
            'action': 'Investigate and potentially retrain model'
        }
    else:
        return {
            'status': 'OK',
            'score': score,
            'message': 'Model fairness within acceptable range'
        }

# Simulate monitoring
result = monitor_model_fairness(
    model,
    (X_test, y_test, gender_test),
    threshold=70
)

print("Production Monitoring Result:")
print(f"Status: {result['status']}")
print(f"Score: {result['score']:.1f}")
print(f"Message: {result['message']}")
if 'action' in result:
    print(f"Action: {result['action']}")

### 5.3 For Compliance Teams: Audit Trail

In [ ]:
# Create compliance report
compliance_report = {
    'audit_date': pd.Timestamp.now().isoformat(),
    'model_type': 'Random Forest Classifier',
    'dataset_size': len(X_test),
    'fairness_threshold': 0.05,
    'overall_score': summary['overall_score'],
    'passes_fairness': summary['passes_fairness'],
    'violations': len(issues),
    'disparate_impact': summary['disparate_impact_ratio'],
    'passes_80_rule': summary['disparate_impact_ratio'] >= 0.80,
    'lgpd_compliant': summary['passes_fairness'],
    'recommendations': [
        issue['message'] for issue in issues
    ] if issues else ['No recommendations - model is fair']
}

# Save compliance report
with open('compliance_audit.json', 'w') as f:
    json.dump(compliance_report, f, indent=2, default=str)

print("✓ Compliance audit saved: compliance_audit.json")
print("\nCompliance Summary:")
print(f"  Audit Date: {compliance_report['audit_date']}")
print(f"  Overall Score: {compliance_report['overall_score']:.1f}/100")
print(f"  Passes Fairness: {compliance_report['passes_fairness']}")
print(f"  Passes 80% Rule: {compliance_report['passes_80_rule']}")
print(f"  LGPD Compliant: {compliance_report['lgpd_compliant']}")

## Part 6: Best Practices for Reporting

### Best Practices Checklist

#### ✅ Before Deploying to Production:
1. Generate comprehensive fairness report
2. Review all fairness violations
3. Document mitigation strategies
4. Get stakeholder approval
5. Save baseline fairness metrics

#### ✅ During Production:
1. Monitor fairness metrics regularly (weekly/monthly)
2. Set up alerts for fairness score drops
3. Compare against baseline metrics
4. Document any fairness drift
5. Retrain when necessary

#### ✅ For Stakeholder Communication:
1. Use HTML reports for visual presentation
2. Focus on overall score and key violations
3. Explain metrics in non-technical terms
4. Provide actionable recommendations
5. Include comparison with industry standards

#### ✅ For Compliance:
1. Archive all fairness audit reports
2. Maintain version history
3. Document threshold justifications
4. Keep audit trail of changes
5. Prepare for regulatory review

## Summary

In this tutorial, you learned:

✅ **Generate reports**: Create comprehensive HTML fairness reports  
✅ **Interpret scores**: Understand overall fairness scores and thresholds  
✅ **Analyze violations**: Identify and address fairness issues  
✅ **Export data**: Save metrics in JSON/CSV for further analysis  
✅ **Compare models**: Evaluate multiple models for best fairness  
✅ **Monitor production**: Set up fairness monitoring pipelines  
✅ **Compliance**: Create audit trails for regulatory requirements  
✅ **Best practices**: Follow guidelines for responsible AI deployment  

## Key Takeaways

1. **Reports are communication tools** - Make them accessible to all stakeholders
2. **Monitor continuously** - Fairness can drift over time
3. **Document everything** - Keep audit trails for compliance
4. **Balance is key** - Consider fairness alongside other metrics
5. **Be proactive** - Address fairness issues before deployment

## Congratulations! 🎉

You've completed all JusticeAI tutorials. You now know how to:
- Perform fairness audits quickly
- Understand all fairness metrics in depth
- Generate and interpret comprehensive reports
- Build responsible ML systems

## Resources

- 📖 [Documentation](https://justiceai-validation.github.io/JusticeAI/)
- 🐛 [Report Issues](https://github.com/JusticeAI-Validation/JusticeAI/issues)
- 💬 [Community Discussions](https://github.com/JusticeAI-Validation/JusticeAI/discussions)

Happy building fair ML systems! 🚀